## Mounting and redirecting to the Dataset

In [1]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [3]:
%cd dataset/

/content/drive/My Drive/Colab Notebooks/dataset


In [4]:
!ls

Youtube01-Psy.csv	 Youtube03-LMFAO.csv
Youtube02-KatyPerry.csv  Youtube04-Eminem.csv


In [5]:
#importing required libraries
import numpy as np
import pandas as pd
from pathlib import Path

In [6]:
df = pd.read_csv('Youtube02-KatyPerry.csv')

In [7]:
df.sample(20)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
80,z13ndjiobv24y14tp230gvzjeovuup1vf,Nicolás Jara,2014-09-10T01:50:55,Click For iTunes code http://shhort.com/a?r=0L...,1
57,z13jtvmwoqjvsxj4p04cdhbwswnsf5qqv24,Ricky Smith,2014-09-07T02:43:55,This song makes me want to drink bleach﻿,0
125,z13nhjbxcwe4stwwz04cfpfoqrzghr540ns0k,Mandeep Kaur,2014-09-16T21:03:17,selfie alert﻿,0
289,z12ltb3iavfkyl4yr04cjperksqvs51otyg0k,Truc Ng,2014-11-04T13:13:28,https://www.facebook.com/myfunnyriddles﻿,1
133,z13jffkgavyhv5tkq22lyboo2subj1vu0,Tushti Chakraborty,2014-09-19T15:30:47,I &lt;3 Katy Perry!﻿,0
271,z12lznkqhkyysbznf222ft35zmy1shkb504,youngj,2014-10-29T11:28:58,People who believe in Illuminati are stupid th...,0
248,z13cynorbtfoydmbx04cfxohsqzvh1jwnlc0k,احمد الهوارى,2014-10-24T09:55:30,(( rapid facebook )) the free version of all c...,1
29,z13ccdmpbuuqc32bf04cj3xrcrrjsxozwtk0k,phemion23,2014-08-21T10:25:48,http://www.mediafire.com/download/yvovhafsv5qz...,1
267,z12izheoltagv1vvg233cvfgpt3yxdhed,NeverGreen35,2014-10-28T20:04:45,Subscribe me and i subscribe you back!!﻿,1
320,z12kgb3b1qbgghhvk23rwj3j5lmehn3fz,Oreja Ramanauskaite,2014-11-08T20:37:45,https://www.tsu.co/ToMeks Go register ;) free ...,1


In [8]:
df.drop(columns=["COMMENT_ID","AUTHOR","DATE"],inplace=True)

In [9]:
df["CONTENT"][4]

'Hey everyone. Watch this trailer!!!!!!!!  http://believemefilm.com?hlr=h2hQBUVB\ufeff'

In [10]:
import html
df["CONTENT"]=df["CONTENT"].apply(html.unescape)
df["CONTENT"]=df["CONTENT"].str.replace("\ufeff","")

In [11]:
df["CONTENT"][4]

'Hey everyone. Watch this trailer!!!!!!!!  http://believemefilm.com?hlr=h2hQBUVB'

In [12]:
df["CONTENT"]=df["CONTENT"].str.replace("(<a.+>)","htmllink")

In [13]:
df[df["CONTENT"].str.contains("<.+>")]["CONTENT"]

31    <script>document.write('htmllink
Name: CONTENT, dtype: object

In [14]:
df["CONTENT"]=df["CONTENT"].str.replace("<.+>","")

In [15]:
df["CONTENT"]=df["CONTENT"].str.replace("\'","")

In [16]:
df["CONTENT"]=df["CONTENT"].str.lower()


In [17]:
df[df["CONTENT"].str.contains("\.com|watch\?")]

,CONTENT,CLASS
0,i love this so much. and also i generate free ...,1
1,http://www.billboard.com/articles/columns/pop-...,1
2,hey guys! please join me in my fight to help a...,1
3,http://psnboss.com/?ref=2tggp3pv6l this is the...,1
4,hey everyone. watch this trailer!!!!!!!! http...,1
...,...,...
318,for latest movies 2014 please visit this site ...,1
327,https://apps.facebook.com/my-polls/utsitcompet...,1
335,curti? click here ? https://www.facebook.com/...,1
339,please vote for me for the wet seal model 2015...,1


In [18]:
df["CONTENT"][173]

'https://vimeo.com/107297364'

In [19]:
df["CONTENT"]=df["CONTENT"].str.replace(r"\S*\.com\S*|\S*watch\?\S*","htmllink")

In [20]:
df["CONTENT"]=df["CONTENT"].str.replace("\W"," ")

In [21]:
df["CONTENT"][3]

'htmllink this is the song'

In [22]:
df["CONTENT"][173]

'htmllink'

## Model Creation

In [23]:
df["CLASS"].value_counts(normalize=True)

1    0.5
0    0.5
Name: CLASS, dtype: float64

In [24]:
vocab=[]
for comment in df["CONTENT"]:
    for word in comment.split():
        vocab.append(word)

In [25]:
vocabulary=list(set(vocab))
len(vocabulary)

1403

In [26]:
# Create a column for each of the unique word in our vocabulary inorder to get the count of words 
for word in vocabulary:
    df[word]=0

In [27]:
df.head()

,CONTENT,CLASS,happen,acoustic,healing,message,stand,official,k,babe,way,card,whats,big,up,nice,paid,reason,encouraging,pleaase,history,6th,00,clear,feedback,2,put,explicit,disabled,nz,fantastic,perrys,decio,ty,fit,install,knowledge,picture,reply,bigger,...,you,mobile,helping,setting,somehow,rip,difference,org,invisible,dissertation,female,hack,update,abused,billie,daniels,subscribers,didnt,3,700,dope,giant,birtgday,house,idiots,flat,foolish,cover,funny,tsu,skinny,evertim,nails,ans,3873,am,dis,producers,pretty,boring
0,i love this so much and also i generate free ...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,htmllink vote for sones please were against...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,hey guys please join me in my fight to help a...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,htmllink this is the song,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hey everyone watch this trailer html...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
# looping through data frame and counting words 
for index,value in enumerate(df["CONTENT"]):
  for l in value.split():
    df[l][index]+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [29]:
df.sample(10)

,CONTENT,CLASS,happen,acoustic,healing,message,stand,official,k,babe,way,card,whats,big,up,nice,paid,reason,encouraging,pleaase,history,6th,00,clear,feedback,2,put,explicit,disabled,nz,fantastic,perrys,decio,ty,fit,install,knowledge,picture,reply,bigger,...,you,mobile,helping,setting,somehow,rip,difference,org,invisible,dissertation,female,hack,update,abused,billie,daniels,subscribers,didnt,3,700,dope,giant,birtgday,house,idiots,flat,foolish,cover,funny,tsu,skinny,evertim,nails,ans,3873,am,dis,producers,pretty,boring
187,who else would give katy perry a good old migh...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
74,please share and help to my friend htmllink ...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
119,see it all human folly right,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
192,i was playing this song and the baby in my bel...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
133,i 3 katy perry,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
86,i think they were drunk when they shot the fir...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,selfie alert,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
158,you are all sheep period this is terrible mu...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
247,attention we need 10 million views more f...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
145,this is the best of the best video in world ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
#Total number of words in each class
df.groupby("CLASS").sum().sum(axis=1)

CLASS
0    2335
1    3295
dtype: int64

In [31]:
# Assign variables to all values required in calculation
p_ham=0.5
p_spam=0.5
n_spam=df[df["CLASS"]==1].drop(columns=["CONTENT","CLASS"]).sum().sum()
n_ham=df[df["CLASS"]==0].drop(columns=["CONTENT","CLASS"]).sum().sum()
n_vocabulary=len(vocabulary)

In [32]:
# Slicing dataframe for each class
df_sspam=df[df["CLASS"]==1]
df_hham=df[df["CLASS"]==0]

In [33]:
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}
for word in vocabulary:
    n_word_given_spam = df_sspam[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + 1) / (n_spam + 1*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    n_word_given_ham = df_hham[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + 1) / (n_ham + 1*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

In [34]:
df["CONTENT"][1]

'htmllink vote for sones please    were against vips    please help us      '

## Model Testing

In [35]:
def classifier(string):
    message=html.unescape(string)
    message=string.replace("\ufeff","")
    message=string.replace("(<a.+>)","htmllink")
    message=string.replace("\'|<.+>","")
    message=string.replace("\S*\.com\S*|\S*watch\?\S*","htmllink")
    message=string.replace("\W"," ").lower()
    p_string_s=1
    p_string_h=1
    for word in message.split():
        if word in parameters_spam:
            p_string_s*=parameters_spam[word]
            p_string_h*=parameters_ham[word]
    if (p_string_s*p_spam)>(p_string_h*p_ham):
        return(1)
    elif (p_string_s*p_spam)<(p_string_h*p_ham):
        return(0)
    else:
        return(-1)

In [36]:
# Reading the dataframe for testing model
df_artist=pd.read_csv("Youtube02-KatyPerry.csv")

In [37]:
df_artist.sample(4)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
163,z13lj1oyrkmrery5o04cjfuitwyagldauxs0k,nailey nicool,2014-09-25T20:24:40,https://www.facebook.com/pages/Nailey-nicool/6...,1
22,z123uflrqpqwzvhts23pfr3jht3ue1kaf04,Mike Music,2014-08-18T14:24:05,"iS IN TOP 10 , IN YOUTUBE TOP VIEWS , ON 9 IS ...",1
211,z12yfvzzpt2tizxvb22yj334vzmrh13gc04,Natalya Korshunova,2014-10-14T03:29:19,Katy Perry - Roar (Official): http://youtu.be/...,0
126,z12nj5gwruilz5a4y04cfdda2wfgz1lwecg,Chris Madzier,2014-09-16T22:50:28,Your going to hear me Roar !!!! :-)))) #soun...,0


In [38]:
df_artist["Pred_Class"]=df_artist["CONTENT"].apply(classifier)

In [39]:
correct_predictions=0
total_rows=0
for row in df_artist.iterrows():
    row=row[1]
    total_rows+=1
    if row["CLASS"]==row["Pred_Class"]:
        correct_predictions+=1
accuracy=correct_predictions/total_rows
print("accuracy=",accuracy)

accuracy= 0.8628571428571429


## Conclusion

In [40]:
classifier("Hello, myself xyxxz i love singing")

0

In [41]:
classifier("Please subscribe to my channel as I'm approaching 1M subscribers")

1

In [42]:
classifier("This is my affiliated link Click on it www.cbybd.com")

1

In [43]:
classifier("This is the all time best album, just awesome!!!")

0

In [45]:
classifier("Contact me http://psnboss.com/?ref=2tggp3pv6l for great offers ")

1